# Assignment 2: ASQL_ICs_DI

Student: Koray Poyraz <br>
Studentnr: 5367646

# Question 1
see file db.sql

Integrity constraints were taken into account when creating the relations.

### Department<br>
Building is not unique because it might be that a building could have multiple departments, so building could have his own relation but due to not having specific attributes related to a building its not needed.

### Student and Instructor<br>
These relations have set null on delete of a tuple in department because we don't want to delete tuples from student and instructor when a department gets deleted, it might be that a student or instructor will join another department.

### Advisor<br>
If a student is deleted or updated then those tuples in the advisor are deleted and updated as well.
If a instructor is deleted then we set the value to null so you can have in 1 glance all students who don't have a advisor. Also, it has cascade on update.

### Course<br>
I didn't make title a unique attribute because it might be that a department wants to give the same course with same title but a different course id.
We could make title and dept_name unique but that would not give the possibility to have the same course with different id, same title within the same department but different credits.

### Prereq<br>
When a course is been deleted or updated then the effect will be cascaded to the tuples in prereq relation.

### Teaches<br>
When we remove a section then there is no need to have the related teaches tuples.

### Section<br>
If there is a new classroom for a section then the update will reflect the change or delete the related tuples on delete.
If a course is been deleted then there is no need to have the related tuples in section.

### Takes<br>
When a student is deleted or updated the change will be reflected on the related tuples in takes.
If a section is deleted than the related tuples in takes will be deleted as well and on update the change will be cascaded.

# Question 2
see file inserts.sql

### - 1st turn on
pragma foreign_keys = on;

### - 2nd add departments
insert into department values ('CS', 'Computer Science', 10000);

insert into department values ('DS', 'Data Science', 10000);

select * from department;

### - Add 4 students to student relation
insert into student (name, dept_name, tot_cred) values ('Jan', 'CS', 10);

insert into student (name, dept_name, tot_cred) values ('Kees', 'DS', 8);

insert into student (name, dept_name, tot_cred) values ('Peter', 'CS', 7);

insert into student (name, dept_name, tot_cred) values ('Klaas', 'DS', 9);

select * from student;

### - Add 4 instructors to instructor relation
insert into instructor (name, dept_name, salary) VALUES ('Daalon', 'CS', 2800);

insert into instructor (name, dept_name, salary) VALUES ('Henk', 'DS', 2800);

insert into instructor (name, dept_name, salary) VALUES ('Jeffrey', 'CS', 3500);

insert into instructor (name, dept_name, salary) VALUES ('Rick', 'DS', 2800);

select * from instructor;

### - Add 4 advisors to advisor relation
insert into advisor (s_id, i_id) VALUES (1, 2);

insert into advisor (s_id, i_id) VALUES (2, 1);

insert into advisor (s_id, i_id) VALUES (3, 4);

insert into advisor (s_id, i_id) VALUES (4, 3);

select * from advisor;

### - Display student names and their advisor name
select s1.name student_name, s2.name advisor_name from advisor s
    inner join student s1 on s.s_id = s1.ID
    inner join instructor s2 on s.s_id = s2.ID;

# Question 3

### R(A,B,C,D,E,F,G)

### Functional dependencies
- ABD -> EG
- C -> DG
- E -> FG
- AB -> C
- G -> F

### Closure
- ABCDEFG+ -> (ABCDEFG)
    - ABD can determine EG so we can discard EG (Reflexivity)
    - and because E and G determines F we can also discard F (Transitivity)
- ABCD+ -> (ABCDEFG)
    - C can determine DG so we can discard DG (Reflexivity)
- ABC+ -> (ABCFEGD)
    - AB can determine C so we can discard C (Reflexivity)
- AB+ -> (ABCDGEF)

The proper subset of AB+ is {A},{B}, A and B are not super key<br>
so we can say that AB is the candidate key.

### Candidate key
- AB

# Question 4

### Functional dependency violation
First name -> Gender

In [1]:
import pandas as pd
df_data = pd.read_csv('../assignment_two/fdExample.csv')

In [3]:
print('Violation of functional dependency:')

df_name_gender = df_data[['First name', 'Gender']].rename(columns={'First name': 'first_name'})

# Version 1: shows for each name a set of different genders (key x values)
# Below I have a code which shows the first name -> genders values which are violated, total is 52 first names
# the code could be more simplified by just using the unique() and intersection() methods
# but I found it important to show both columns values

violated_values = df_name_gender\
    .groupby('first_name')\
    .apply(lambda x: ','.join(x['Gender']))\
    .reset_index()\
    .rename(columns={0:'count_diff_gender'})

count_names = 0
for row in violated_values.values:
    set_val = set(row[1].split(','))
    if len(set_val) > 1:
        count_names += 1
        # currently I only need count_names as extra info together with version 2
        # but (see latest code line of this code cell)
        # you can uncomment the code line below to see the unique names and
        # their related multi genders printed
        # print(f' First name: {row[0]} \t - Gender: {set_val}')

# print(f'Violated: {count_names}')


# Version 2: shows all tuples including the row number, the name and gender which violate
key_map = {}

for index, x in enumerate(df_name_gender.values):
    if x[0] not in key_map:
        key_map[x[0]] = [f'{index}:{x[1]}']
    else:
        idx_of_name = key_map[x[0]]
        idx_of_name.append(f'{index}:{x[1]}')

count = 0

for name in key_map:
    genders = map(lambda x: x.split(':')[1], key_map[name])
    is_violated = len(set(genders)) > 1

    if is_violated:
        for gender in key_map[name]:
            count += 1
            gender_idx = gender.split(':')
            print(f'Row number: [{gender_idx[0]}] \t name: {name} \t gender: {gender_idx[1]}')

        print('--------------------------------------------------------------------')

print(f'Number of first names which violate:\t {count_names}')
print(f'Number of tuples which violate:\t {count}')

Violation of functional dependency:
Row number: [1] 	 name: David 	 gender: M
Row number: [144] 	 name: David 	 gender: M
Row number: [187] 	 name: David 	 gender: M
Row number: [201] 	 name: David 	 gender: M
Row number: [202] 	 name: David 	 gender: M
Row number: [203] 	 name: David 	 gender: M
Row number: [300] 	 name: David 	 gender: M
Row number: [326] 	 name: David 	 gender: M
Row number: [346] 	 name: David 	 gender: M
Row number: [357] 	 name: David 	 gender: M
Row number: [396] 	 name: David 	 gender: M
Row number: [460] 	 name: David 	 gender: M
Row number: [546] 	 name: David 	 gender: M
Row number: [601] 	 name: David 	 gender: M
Row number: [652] 	 name: David 	 gender: M
Row number: [653] 	 name: David 	 gender: M
Row number: [768] 	 name: David 	 gender: M
Row number: [799] 	 name: David 	 gender: M
Row number: [845] 	 name: David 	 gender: M
Row number: [874] 	 name: David 	 gender: M
Row number: [881] 	 name: David 	 gender: M
Row number: [917] 	 name: David 	 gender: 

# Question 5

## Metanome

For this question I used the Metanome tool together with the FastFDs-1.2 and DepMiner-1.2 algorithm.

For FastFD (19 results), please see fd_algorithm_1.png and fd_algorithm_2.png

|determinant|dependant|
|---|:---:|
|Abbreviation|Name|
|Unicode|Name|
|UnicodeDisplay|Name|
|Represents|Name|
|Name|AU_Abbreviation|
|Unicode|AU_Abbreviation|
|UnicodeDisplay|AU_Abbreviation|
|Represents|AU_Abbreviation|
|Name|Symbol|
|AU_Abbreviation|Symbol|
|Unicode|Symbol|
|UnicodeDisplay|Symbol|
|Represents|Symbol|
|UnicodeDisplay|Unicode|
|Represents|Unicode|
|Unicode|UnicodeDisplay|
|Represents|UnicodeDisplay|
|Unicode|Represents|
|UnicodeDisplay|Represents|

For DepMiner (14 results), please see dm_algorithm_1.png and dm_algorithm_2.png

|determinant|dependant|
|---|:---:|
|IAU_Abbreviation|Name |
|UnicodeDisplay|Name |
|Represents|	Name|
|Unicode|	Name|
|Represents|	UnicodeDisplay|
|Unicode|	UnicodeDisplay|
|UnicodeDisplay|	IAU_Abbreviation|
|Represents|	IAU_Abbreviation|
|Name|	IAU_Abbreviation|
|Unicode|	IAU_Abbreviation |
|UnicodeDisplay|	Unicode|
|Represents|	Unicode|
|UnicodeDisplay|	Represents|
|Unicode|	Represents|

Algorithm FastFD (19 results) has more results than DepMiner (14 results).

# Question 6

## Computing Levenshtein distance

'*' = empty space

'+' = shows the minimum cost path

# 1

#### Computed
| .|.	 |	C|	o|	m|	p|	l|	e|	t|	i|	o|	n|
|----|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|. |+0|	1|	2|	3|	4|	5|	6|	7|	8|	9|	10|
|C|	1|	+0|	1|	2|	3|	4|	5|	6|	7|	8|	9|
|o|	2|	1|	+0|	1|	2|	3|	4|	5|	6|	7|	8|
|m|	3|	2|	1|	+0|	1|	2|	3|	4|	5|	6|	7|
|p|	4|	3|	2|	1|	+0|	1|	2|	3|	4|	5|	6|
|u|	5|	4|	3|	2|	+1|	1|	2|	3|	4|	5|	6|
|t|	6|	5|	4|	3|	2|	+2|	2|	2|	3|	4|	5|
|a|	7|	6|	5|	4|	3|	3|	+3|	3|	3|	4|	5|
|t|	8|	7|	6|	5|	4|	4|	4|	+3|	4|	4|	5|
|i|	9|	8|	7|	6|	5|	5|	5|	4|	+3|	4|	5|
|o|	10|	9|  8|	7|	6|	6|	6|	5|	4|	+3|	4|
|n|	11|	10|	9|	8|	7|	7|	7|	6|	5|	4|	+3|

result = 3 cost

#### Handwritten
- Computation
- Comp*letion

d+s+s = 3 operations = 3 cost

# 2

#### Computed

| .|.	 |	C|	o|	m|	p|	l|	e|	t|	i|	o|	n|
|----|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| .|+0|	1|	2|	3|	4|	5|	6|	7|	8|	9|	10|
|C|	1|	+0|	1|	2|	3|	4|	5|	6|	7|	8|	9|
|o|	2|	1|	+0|	1|	2|	3|	4|	5|	6|	7|	8|
|m|	3|	2|	1|	+0|	1|	2|	3|	4|	5|	6|	7|
|p|	4|	3|	2|	1|	+0|	1|	2|	3|	4|	5|	6|
|u|	5|	4|	3|	2|	+1|	2|	3|	4|	5|	6|	7|
|t|	6|	5|	4|	3|	2|	+3|	4|	3|	4|	5|	6|
|a|	7|	6|	5|	4|	3|	4|	+5|	4|	5|	6|	7|
|t|	8|	7|	6|	5|	4|	5|	6|	+5|	6|	7|	8|
|i|	9|	8|	7|	6|	5|	6|	7|	6|	+5|	6|	7|
|o|	10|	9|	8|	7|	6|	7|	8|	7|	6|	+5|	6|
|n|	11|	10|	9|	8|	7|	8|	9|	8|	7|	6|	+5|

result = 5 cost

#### Handwritten
- Computation
- Comp*letion

d+s+s = 3 operations = i + (2 x 2) = 5 cost

# 3

#### Computed

| .|.|	C|	o|	m|	p|	l|	e|	t|	i|	o|	n|
|----|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|.| +0|	1|	2|	3|	4|	5|	6|	7|	8|	9|	10|
|C|	1|	+0|	1|	2|	3|	4|	5|	6|	7|	8|	9|
|o|	2|	1|	+0|	1|	2|	3|	4|	5|	6|	7|	8|
|m|	3|	2|	1|	+0|	1|	2|	3|	4|	5|	6|	7|
|p|	4|	3|	2|	1|	+0|	+1|	+2|	3|	4|	5|	6|
|u|	5|	4|	3|	2|	1|	2|	+3|	4|	5|	6|	7|
|t|	6|	5|	4|	3|	2|	3|	+4|	3|	4|	5|	6|
|a|	7|	6|	5|	4|	3|	4|	+5|	4|	5|	6|	7|
|t|	8|	7|	6|	5|	4|	5|	6|	+5|	6|	7|	8|
|i|	9|	8|	7|	6|	5|	6|	7|	6|	+5|	6|	7|
|o|	10|	9|	8|	7|	6|	7|	8|	7|	6|	+5|	6|
|n|	11|	10|	9|	8|	7|	8|	9|	8|	7|	6|	+5|

result = 5 cost

#### Handwritten
- Computation
- Comp*letion

d+s+s = 3 operations = i + (2 x 3) = 7 cost

(this is the result if we keep the empty space * between p and l of completion)
<br>
<br>
- Comp**utation <br>
- Comple***tion <br>

i+i+d+d+d = 5 operations = (i x 1) + (d x 1) = 5 cost

(this is the result if we add more empty space to use less cost)

------
When the update cost is > 3: <br>
The algorithm is looking for the minimum cost out of the edit distances (insert, substitute, deletion) from the current position [i,j].<br>
So, if the cost for substitution is high but the cost for e.g. insert/ delete in the edit distances at [i,j] position is lower, than insert/ delete operation is applied.

# Question 7

## Computing the gap distance

- (i) insertion cost = 1
- (o) open gap cost = 1
- (e) extend gap cost = 0.1

<p>'Adv ances in Instrum entation and Control'</p>
<p>'Adv._________ Instrum._____________ Control'</p>
<p>'Adv[i][o][e][e][e][e][e][e][e][e] Instrum[i][o][e][e][e][e][e][e][e][e][e][e][e][e] Control'</p>

- i = 1 + 1 = 2
- o = 1 + 1 = 2
- e = 8 + 12 = 20

cost = i+o+e = 2 + 2 + (20 x 0.1) <br>
cost = 24

# Question 8

## Compute Jaccard bag similarity and Jaccard Distance

A = {1,1,2,2,5}; B = {1,2,2,2,5,5}; C = {1,2,3,4,5}

A and B <br>
- J bag sim  = 4/11 = 0.3636
- J distance = 7/11 = 0.6363

A and C <br>
- J bag sim  = 3/10 = 0.30
- J distance = 7/10 = 0.70

B and C <br>
- J bag sim  = 3/11 = 0.2727
- J distance = 8/11 = 0.7272

# Question 9


## Compute the Jaro and Jaro-Winkler similarity

- a r n a b
- u r b a n

### Jaro sim <br>
- c=2
- t=0
- l=5

0.6 = (1/3) * ((2/5)+(2/5)+((2-0)/2))
<br>
<br>
### Jaro Winkler <br>
- P=0.1
- L=0

Jw(S1, S2) = JaroSim + P x L x (1-JaroSim)

0.6 = 0,6 + 0,1 x 0 x (0,4)

They give the same result due to not having common prefix in both strings.
If e.g. they would have a common prefix then Jaro Winkler would give a better matching result.

# Question 10
## Cardinality of the set that has 5-shingles

In [208]:
sentence = 'Many problems can be expressed as finding similar sets'
k = 5

cardinality_result = len(sentence) + 1 - k

print(f'A set with {k}-shingles has {cardinality_result} cardinality')

A set with 5-shingles has 50 cardinality


# Question 11

- D1 = {aa, bb, ab , ba}
- D2 = {aa, ac, ca, ba}
- D3 = {ab, ba, ca}

# a - matrix representation
matrix representation of shingles documents relationship, the universal set is alphabetically ordered <br>

| . | D1 | D2 | D3 |
|----|:---:|:---:|:---:|
|aa|1|1|0|
|ab|1|0|1|
|ac|0|1|0|
|ba|1|1|1|
|bb|1|0|0|
|ca|0|1|1|

# b - signature matrix

Permutation 1

| index | P1 | D1 | D2 | D3 |
|----|:---:|:---:|:---:|:---:|
|1|aa|1|1|0|
|2|bb|1|0|0|
|3|ab|1|0|1|
|4|ba|1|1|1|
|5|ac|0|1|0|
|6|ca|0|1|1|

Permutation 2

| index | P2 | D1 | D2 | D3 |
|----|:---:|:---:|:---:|:---:|
|1|ca|0|1|1|
|2|ac|0|1|0|
|3|ba|1|1|1|
|4|ab|1|0|1|
|5|bb|1|0|0|
|6|aa|1|1|0|

Permutation 3

| index | P3 | D1 | D2 | D3 |
|----|:---:|:---:|:---:|:---:|
|1|ac|1|1|0|
|2|ca|1|0|1|
|3|ab|0|1|0|
|4|ba|1|1|1|
|5|bb|1|0|0|
|6|aa|0|1|1|

signature matrix

| . | .   | . | . |
|----|:---:|:---:|:---:|
|P1|1|1|3|
|P2|3|1|1|
|P3|1|1|2|

# c - Jaccard similarity
Comparison of pair documents with their signatures by using Jaccard similarity

| . | (D1, D2)   | (D1, D3) | (D2, D3) |
|----|:---:|:---:|:---:|
|Col/Col|2/6 = 0,33|2/5 = 0,4|2/5 = 0,4|
|Sig/Sig|2/3 = 0,66|0/3 = 0|1/3 = 0,33|

Info on col and sig:
- D1 and D2: on 'Col' similarity is 33% and on 'Sig' the agreement is 66%.
- D1 and D3: on 'Col' similarity is 40% and on 'Sig' the agreement is 0%.
- D2 and D3: on 'Col' similarity is 40% and on 'Sig' the agreement is 33%.